In [1]:

# Set up notebook
%pprint
import sys
if (osp.join('..', 'py') not in sys.path): sys.path.insert(1, osp.join('..', 'py'))
from FRVRS import (nu, fu, re)
nu.delete_ipynb_checkpoint_folders()

Pretty printing has been turned OFF


In [2]:

# Extract all the needed columns from all the function calls in the notebook
import inspect
import pyperclip

file_path = '../open_world/Eighteen Variables.ipynb'
df_name = 'csv_stats_df'
functions_regex = re.compile(r'\bfu\.[a-z_]+\([^\)]+_df[^\)]*\)')
issubset_regex = re.compile(r'\.issubset\(')
statements_list = ['These functions still need code to ensure all needed columns are present in the DataFrame:']
needed_columns_set = set()
with open(file_path, 'r', encoding=nu.encoding_type) as f:
    text = f.read()
    parts_list = functions_regex.findall(text)
    for fn_str in parts_list:
        fn_obj = eval(fn_str.split('(')[0])
        source_code = inspect.getsource(fn_obj)
        if issubset_regex.search(source_code) is None:
            statements_list.append('    ' + fn_obj.__name__)
        else:
            parts_list = re.split('(needed_columns = |all_columns = )', source_code, 0)
            needed_columns_str = parts_list[2].strip()
            try:
                needed_columns_set = needed_columns_set.union(eval(re.sub(r'\bself\.', 'fu.', needed_columns_str)))
                # print(needed_columns_set)
            except:
                # print(needed_columns_str)
                continue
if len(statements_list) > 1: print('\n'.join(statements_list))
elif len(needed_columns_set):
    print_str = f"""
# Ensure all needed columns are present in {df_name}
needed_columns = set({sorted(needed_columns_set)})
all_columns = set({df_name}.columns)
assert needed_columns.issubset(all_columns), f"You're missing {{needed_columns.difference(all_columns)}} from {df_name}"
"""
    pyperclip.copy(print_str)
    print(print_str)


# Ensure all needed columns are present in csv_stats_df
needed_columns = set(['action_tick', 'action_type', 'injury_id', 'injury_record_required_procedure', 'injury_severity', 'injury_treated_injury_treated', 'injury_treated_injury_treated_with_wrong_treatment', 'injury_treated_required_procedure', 'location_id', 'patient_engaged_salt', 'patient_id', 'patient_record_salt', 'patient_salt', 'patient_sort', 'scene_id', 'session_uuid', 'tool_applied_type'])
all_columns = set(csv_stats_df.columns)
assert needed_columns.issubset(all_columns), f"You're missing {needed_columns.difference(all_columns)} from csv_stats_df"

